In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from skimage import io
import os
from tqdm import tqdm_notebook
from PIL import Image
from PIL.TiffTags import TAGS
import matplotlib.pyplot as plt
%matplotlib inline
%cd /media/data/DeepFLaSH

In [ ]:
from scipy import ndimage
from skimage.feature import peak_local_max
from skimage.segmentation import relabel_sequential
from skimage.measure import regionprops_table, label

# https://scikit-image.org/docs/stable/api/skimage.measure.html?highlight=measure%20perimeter#skimage.measure.regionprops
def measure_rois_io(mask, image,file_id, threshold=0.5, min_pixel=30, 
                   properties = ['mean_intensity', 
                                 #'max_intensity', 
                                 #'min_intensity', 
                                 #'filled_image',
                                 'label',
                                 #'centroid',
                                 'area',
                                 #'bbox'
                                ]):
    if mask.ndim == 3:
        mask = np.squeeze(mask, axis=2)

    # apply threshold to mask
    # bw = closing(mask > threshold, square(2))
    bw = (mask > threshold).astype(int)

    # label image regions
    label_image = label(bw, connectivity=2) # Falk p.13, 8-“connectivity”.

    # remove areas < min pixel   
    unique, counts = np.unique(label_image, return_counts=True)
    label_image[np.isin(label_image, unique[counts<min_pixel])] = 0
    
    # re-label image
    label_image, _ , _ = relabel_sequential(label_image, offset=1)  
    
    # measure region props
    if label_image.max()>0:
        props_inner = regionprops_table(label_image, image, properties=properties)
        df = pd.DataFrame(props_inner)
    else:
        df = pd.DataFrame(np.nan,index=[0],columns=properties)
         
    df['Nummer'] = file_id 
    return(df)

## Global Settings

In [ ]:
CHANNELS_IMG = 1
MASK_FOLDER = 'pred_cv'
DATA_FOLDER = 'data_labs'
IMAGE = 'red'
NEUN = 'NeuN'

# Labs

### Lab Inns_01

In [ ]:
LAB = 'inns_01'
MASK = 'cFOS'
MIN_PIXEL = 16
META = None

### Lab Inns_02

In [ ]:
LAB = 'inns_02'
MASK = 'cFOS'
MIN_PIXEL = 60
META = 'data_labs/inns_02/_metadata_inns_02.tif'

### Lab Mue

In [ ]:
LAB = 'mue'
MIN_PIXEL = 30
MASK = 'cFOS'
META = None

### Lab Wue_02

In [ ]:
LAB = 'wue_02'
MASK = 'Parv'
IMAGE = 'green'
MIN_PIXEL = 112
META = 'data_labs/wue_02/images_test/1674_green.tif'

### Load and adjust data

Load Masks

In [ ]:
data_path = DATA_FOLDER+'/{}/images_test'.format(LAB)
file_ids = [each.rsplit('_',1)[0] for each in os.listdir(data_path) if each.endswith(IMAGE + '.tif')]
image_list = [io.imread(os.path.join(data_path, img_name), as_gray=True) for 
              img_name in [s + '_' + IMAGE + '.tif' for s in file_ids]]
neun_list = [io.imread(os.path.join(data_path, img_name), as_gray=True) for 
              img_name in [s + '_' + NEUN + '.tif' for s in file_ids]]
len(image_list)

Calc NeuN Area

In [ ]:
df_neun_list = []
df_neun_list = [measure_rois_io(msk, img, fid) for msk, img, fid in zip(neun_list, image_list, file_ids)]
df_neun = pd.concat(df_neun_list)
df_neun['Neun_Area'] = df_neun['area']
#df_neun['Neun_Area_μ^2'] = df_neun['area']/conv_factor**2
#df_neun['Neun_Area_mm^2'] = df_neun['area']/1000**2   #Area entspricht nun mm²
df_neun.to_csv(os.path.join('_bio_eval',LAB,LAB+'_'+NEUN+'_results.csv'), index=False)

In [ ]:
df_neun.head()

In [ ]:
mask_path = MASK_FOLDER+'/{}/test/binary/'.format(LAB) #data/labels'
ens_list = [x for x in next(os.walk(mask_path))[1] if not x.startswith('.')]
ens_list

### Loop over Models

In [ ]:
df_cfos_list = []
for ens in tqdm_notebook(ens_list):
    fold_list = [x for x in next(os.walk(os.path.join(mask_path, ens)))[1] if not x.startswith('.')]
    model_list = [[x for x in next(os.walk(os.path.join(mask_path, ens, fold)))[1] if not x.startswith('.')][0] for fold in fold_list]
    for fold, model in zip(fold_list, model_list):
        model_path = os.path.join(mask_path, ens, fold, model)
        mask_list = [io.imread(os.path.join(model_path, x), as_gray=True).astype('int') for x in [s + '_' + MASK + '.png' for s in file_ids]]
        for msk, neun in zip(mask_list, neun_list):
            msk[neun==0]=0
        df_list = [measure_rois_io(msk, img, fid, min_pixel=MIN_PIXEL) for msk, img, fid in zip(mask_list, image_list, file_ids)]
        df_tmp = pd.concat(df_list)
        df_tmp['fold'] = fold
        df_tmp['unet'] = model
        df_tmp['ens'] = ens
        df_cfos_list += [df_tmp]
df_cfos = pd.concat(df_cfos_list)

In [ ]:
df_cfos.head()

In [ ]:
df_cfos.to_csv(os.path.join('_bio_eval',LAB,LAB+'_'+MASK+'_ROIs.csv'), index=False)

### Check results

In [ ]:
df_cfos[(df_cfos.Nummer=='1048') & (df_cfos.unet=='f27c_sc_cyc_mue_cFOS_staple_5.0010.h5')]#.groupby(['Nummer', 'unet']).agg(['mean', 'count'])

In [ ]:
df_cfos.groupby(['Nummer', 'unet']).agg(['mean', 'count'])

# Old

In [ ]:
conv_factor = 1.
if META is not None:
    img = Image.open(META)
    meta_dict = {TAGS[key] : img.tag[key] for key in img.tag.keys()}
    conv_factor = meta_dict['XResolution'][0][0]/meta_dict['XResolution'][0][1]
print(conv_factor)

## Pad NeuN Mask

In [ ]:
h_add = (image_list[0].shape[0]-neun_list[0].shape[0])//2
w_add = (image_list[0].shape[1]-neun_list[0].shape[1])//2
neun_list = [np.pad(n, ((h_add, h_add),(w_add, w_add)), 'constant', constant_values=0) for n in neun_list]